# Programming Project - Unit 2,4
*by Igor A. Brandão and Leandro Max*

**Goals**
The purpose of this project is explore the following:

- Full content of the statistical part seen in the course;
- Graphs generation;
- Geolocation analysis and hypotheses should be explained in detail.

## Global Imports

Import the necessary libraries to handle 

- Geocoding;
- Maps;
- File input;
- Heatmap;
- Bokeh charts;
- Numpy library;
- Tqdm progress bar

In [ ]:
### Library necessary to run this IPython Notebook
!pip install geocoder
!pip install folium
!pip install tqdm

In [79]:
# Import pandas
import pandas as pd

# Import google geocoder
import geocoder as gc

# Import numpy library
import numpy as np

# Import folium heatmap
import folium
from folium.plugins import HeatMap

# Import tqdm progressing bar plugin
from tqdm import tqdm

# Import bokeh libraries
from bokeh.charts import Bar, Histogram, Donut, BoxPlot, output_notebook, show
from bokeh.layouts import row
from bokeh.models import HoverTool
from bokeh.charts.attributes import cat, color
from bokeh.charts.operations import blend

## Data importing

This cell is responsible to import the dataSet from excel file and save it into a variable **xl**

In [62]:
# Import pandas
import pandas as pd

# Assign spreadsheet filename: file
file = 'imd_student_blind.xlsx'

# Load spreadsheet: xl
xl = pd.ExcelFile(file)

# Print sheet names
print(xl.sheet_names)

['Sheet1']


## Data printing

Here the dataFrame receive the dataSet from **Sheet1** --> *xl.parse(0)*

In [63]:
# Load a sheet into a DataFrame by index: df
df = xl.parse(0)

# Print the head of the DataFrame df
df.head()

,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina
0,0,59015430,2014,1,CANCELADO,2014,2,2.6,0,Reprovado
1,0,59015430,2014,1,CANCELADO,2015,1,8.0,0,Aprovado
2,1,59073120,2014,1,CANCELADO,2014,2,0.1,0,Reprovado
3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado
4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado


In [27]:
df.columns

Index(['a_ID', 'CEP', 'ano_ingresso', 'periodo_ingresso', 'status',
       'ano_disciplina', 'periodo_disciplina', 'nota', 'disciplina_ID',
       'status.disciplina'],
      dtype='object')

In [28]:
df.shape

(4842, 10)

## 1) Geolocation handler section

Here in this section, we'll handle the geolocalization infos. The idea is converting the zipcode into latitude and longitude and export the new dataSet (as it requires a long time to perform this operation).

After that, it'll be possible to pin the students position and generate the heatMap.

In [29]:
# df["GeoCod"] = df["CEP"]
df['lat'], df['long'] = [0, 0]

# Print new df
df.head()

,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,lat,long
0,0,59015430,2014,1,CANCELADO,2014,2,2.6,0,Reprovado,0,0
1,0,59015430,2014,1,CANCELADO,2015,1,8.0,0,Aprovado,0,0
2,1,59073120,2014,1,CANCELADO,2014,2,0.1,0,Reprovado,0,0
3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado,0,0
4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado,0,0


### *Warning: Do not process this cell again!*

We've already converted all CEP into *lat/long*, you just need to skip this cell.

Please, use the **py-students-blind-with-lat-long.csv** file to generate the maps and save your time and your processor ;)

In [34]:
# Retrieve the latitude and longitute related to each student
for i in tqdm(range(len(df))):
    str = df.loc[i,'CEP']
    g = gc.google(str)
    if g.lat != None:
        df.ix[i,'lat'] = g.lat
        df.ix[i,'long'] = g.lng
print('Geocoding complete!')


100%|██████████| 4842/4842 [48:14<00:00,  2.18it/s] 

Geocoding complete!


In [33]:
# Print df with latitude and longitude
df.head()

,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,lat,long
0,0,59015430,2014,1,CANCELADO,2014,2,2.6,0,Reprovado,-5.816641,-35.200015
1,0,59015430,2014,1,CANCELADO,2015,1,8.0,0,Aprovado,-5.816641,-35.200015
2,1,59073120,2014,1,CANCELADO,2014,2,0.1,0,Reprovado,-5.853337,-35.252804
3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado,-5.832998,-35.242542
4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado,-5.872282,-35.206600


### Latitude and longitude export

In order to avoid unnecessary processing, we are exporting the data *lat/long* from google API to a .csv file.

In [38]:
# Export the new dataSet to csv (don't run this cell again)
df.to_csv('py-students-blind-with-lat-long.csv', encoding="utf-8")

### Please, proceed from here :)

In [39]:
# Read the generate csv
geodata1 = pd.read_csv('py-students-blind-with-lat-long.csv', encoding="utf-8", index_col=0)

In [40]:
# Retrieve the data related to geolocalization
geodata = geodata1.filter(['a_ID','CEP','lat','long'], axis=1)
geodata = geodata.rename(columns = {'a_ID':'Aluno'})

# Reset the index
geodata = geodata.reset_index()

# Remove the the previous index
geodata.drop(geodata.columns[0], axis=1, inplace=True)
geodata.head()

,Aluno,CEP,lat,long
0,0,59015430,-5.816641,-35.200015
1,0,59015430,-5.816641,-35.200015
2,1,59073120,-5.853337,-35.252804
3,2,59072580,-5.832998,-35.242542
4,3,59088150,-5.872282,-35.206600


In [41]:
geodata.columns

Index(['Aluno', 'CEP', 'lat', 'long'], dtype='object')

In [42]:
geodata.shape

(4842, 4)

### Pin map

The idea here is generating a map with pinnings indicating the student location

In [43]:
# Set map center and zoom level
mapc = [-5.788, -35.202]
zoom = 11

# Create map object
map_osm = folium.Map(location=mapc, zoom_start=zoom)

# Plot each of the locations that we geocoded
for j in tqdm(range(len(geodata))):
    folium.Marker([geodata.ix[j,'lat'], geodata.ix[j,'long']],
                  #popup=(geodata.ix[j,'Aluno'])
                 ).add_to(map_osm)
# Show the map
map_osm

100%|██████████| 4842/4842 [01:12<00:00, 66.99it/s]


### Heatmap map

In order to see in a properly way the students concentration in Natal, we'll generate a hetmap to demonstrate that using colors.

Cold colors represent low concentration of students, and hot colors indicate high concentrations.

In [44]:
# Do some fix in final data
dataFinal = geodata.copy()
dataFinal["Count"] = 0

# Print the dataSet head
dataFinal.head()

,Aluno,CEP,lat,long,Count
0,0,59015430,-5.816641,-35.200015,0
1,0,59015430,-5.816641,-35.200015,0
2,1,59073120,-5.853337,-35.252804,0
3,2,59072580,-5.832998,-35.242542,0
4,3,59088150,-5.872282,-35.206600,0


In [45]:
# Cound the number of students by zipcode
dataFinal = pd.DataFrame(dataFinal.groupby(["CEP"])['Count'].count()).reset_index()

# Add latitude and longitude to dataFinal
dataFinal["lat"] = geodata['lat']
dataFinal["long"] = geodata['long']

# Print the dataSet head
dataFinal.head()

,CEP,Count,lat,long
0,0,289,-5.816641,-35.200015
1,1507000,6,-5.816641,-35.200015
2,5021000,6,-5.853337,-35.252804
3,5163000,7,-5.832998,-35.242542
4,6321200,9,-5.872282,-35.206600


In [72]:
dataFinal.shape

(627, 4)

In [46]:
# Set map center and zoom level
mapc = [-5.788, -35.202]
zoom = 11

# Initialize the coordinates array
coordinates = []

# Add the coordinates to the coordinate
for i in range(len(dataFinal)):
    # eliminate items with'nan' element
    if all(~np.isnan([dataFinal.ix[i,'lat'], dataFinal.ix[i,'long'], dataFinal.ix[i,'Count']])):
        coordinates.append([dataFinal.ix[i,'lat'], dataFinal.ix[i,'long'], dataFinal.ix[i,'Count']])

# Create map object
htMap = folium.Map(location=mapc, zoom_start=zoom)

# Append the coordinates to the heatMap
HeatMap(coordinates).add_to(htMap)

# Print the heatMap
htMap

## 2) Statistic handler section

Here in this section, we'll handle the statistics infos.

### Grades histogram

Here we'll analyse the grades taking into consideration its distribution

In [64]:
# Tools
TOOLS = 'box_zoom,box_select,crosshair,resize,reset,hover,save'

# Make the Histogram: p
hist_grade = Histogram(df, 'nota', title='Students grades distribution', 
              legend='top_left', tools=TOOLS, bins=50, 
                       background_fill_color="#E8DDCB", color="#036564")

# Set axis labels
hist_grade.xaxis.axis_label = 'Grades (0 to 10)'
hist_grade.yaxis.axis_label = 'Grades frequency'

# Call the output_notebook() 
output_notebook()
show(hist_grade)

Loading BokehJS ...

In [65]:
# Imports
import numpy as np
import scipy.special

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_notebook

# Tools
TOOLS = 'box_zoom,box_select,crosshair,resize,reset, hover, save'

p1 = figure(title="Student grades - Normal Distribution (μ=0, σ=0.5)",tools=TOOLS,
            background_fill_color="#E8DDCB")

# Confidence interval
mu, sigma = 0, 0.5

# Histogram settings
hist, edges = np.histogram(df['nota'], density=True, bins=50)

# Tendency line settings
x = np.linspace(-5, 10, 1000)

# Probability density function
pdf = 1/(sigma * np.sqrt(2*np.pi)) * np.exp(-(x-mu)**2 / (2*sigma**2))

# Cumulative density function
cdf = (1+scipy.special.erf((x-mu)/np.sqrt(2*sigma**2)))/2

p1.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
        fill_color="#036564", line_color="#033649")

# Add the lines
p1.line(x, pdf, line_color="#D95B43", line_width=8, alpha=0.7, legend="Probability density function")
p1.line(x, cdf, line_color="white", line_width=2, alpha=0.7, legend="Cumulative density function")

# Customs
p1.legend.location = "top_left"
p1.xaxis.axis_label = 'Grades (0 to 10)'
p1.yaxis.axis_label = 'Grades frequenc)'

# Print the distribution
output_notebook()
show(gridplot(p1, ncols=2, plot_width=800, plot_height=600, toolbar_location=None))

Loading BokehJS ...

In [173]:
df.head()

,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,lat,long,logradouro,bairro,cidade,uf
0,0,59015430,2014,1,CANCELADO,2014,2,2.6,0,Reprovado,NaN,NaN,Avenida Xavier da Silveira,Tirol,Natal,RN
1,0,59015430,2014,1,CANCELADO,2015,1,8.0,0,Aprovado,NaN,NaN,Avenida Xavier da Silveira,Tirol,Natal,RN
2,1,59073120,2014,1,CANCELADO,2014,2,0.1,0,Reprovado,NaN,NaN,Rua Santo Onofre,Planalto,Natal,RN
3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado,NaN,NaN,Rua Ivo Furtado,Cidade Nova,Natal,RN
4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado,NaN,NaN,Rua Itapecirica,Neópolis,Natal,RN


In [83]:
from bokeh.palettes import RdBu3

# Receiva data from dataFrame
data = df.copy()

# Tools
TOOLS = 'box_zoom,box_select,crosshair,resize,reset,hover,save'

# Make a box plot: p
p = BoxPlot(data, values='nota', label='status.disciplina',
            title='Student grades (grouped by discipline status)',
            legend='bottom_right', background_fill_color="#E8DDCB",
            color=RdBu3, plot_width=600, plot_height=600, tools=TOOLS)

# Set the y axis label
p.yaxis.axis_label='Student grades'

# Call the output_notebook() 
output_notebook()
show(p)


Loading BokehJS ...

In [84]:
from bokeh.palettes import RdBu3

# Receiva data from dataFrame
data = df.copy()

# Tools
TOOLS = 'box_zoom,box_select,crosshair,resize,reset,hover,save'

# Make a box plot: p
p = BoxPlot(data, values='nota', label='status',
            title='Student grades (grouped by discipline status)',
            legend='bottom_right', background_fill_color="#E8DDCB",
            color=RdBu3, plot_width=600, plot_height=600, tools=TOOLS)

# Set the y axis label
p.yaxis.axis_label='Student grades'

# Call the output_notebook() 
output_notebook()
show(p)


Loading BokehJS ...

### Student status analysis

The idea here is analyse the (%) between approvals and desaprovals

In [66]:
# Do some fix in studentStatus
studentStatus = df.copy()
studentStatus["Count"] = 0

# Print the dataSet head
studentStatus.head()

,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,Count
0,0,59015430,2014,1,CANCELADO,2014,2,2.6,0,Reprovado,0
1,0,59015430,2014,1,CANCELADO,2015,1,8.0,0,Aprovado,0
2,1,59073120,2014,1,CANCELADO,2014,2,0.1,0,Reprovado,0
3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado,0
4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado,0


In [67]:
# Count the status sum-up
studentStatus = pd.DataFrame(studentStatus.groupby(["status.disciplina"])['Count'].count()).reset_index()

# Print the dataSet head
studentStatus.head()

,status.disciplina,Count
0,Aprovado,2766
1,Reprovado,2076


In [68]:
from bokeh.palettes import RdBu3

# Tools
TOOLS = 'box_zoom,box_select,crosshair,resize,reset,hover,save'

# Donut chart settings
d = Donut(studentStatus, label=['status.disciplina', 'Count'], values='Count',
          text_font_size='12pt', hover_text='status_count', legend='top_left', 
          tools=TOOLS, background_fill_color="#E8DDCB", title='Total approvals and desaprovals', 
          color=RdBu3)

# Print the chart
output_notebook()
show(d)

Loading BokehJS ...

### Discipline status analysis

Here we'll check the (%) of students that quit, cancel or get through the discipline

In [69]:
# Do some fix in disciplineStatus
disciplineStatus = df.copy()
disciplineStatus["Count"] = 0

# Print the dataSet head
disciplineStatus.head()

,a_ID,CEP,ano_ingresso,periodo_ingresso,status,ano_disciplina,periodo_disciplina,nota,disciplina_ID,status.disciplina,Count
0,0,59015430,2014,1,CANCELADO,2014,2,2.6,0,Reprovado,0
1,0,59015430,2014,1,CANCELADO,2015,1,8.0,0,Aprovado,0
2,1,59073120,2014,1,CANCELADO,2014,2,0.1,0,Reprovado,0
3,2,59072580,2014,1,ATIVO,2014,2,6.1,0,Aprovado,0
4,3,59088150,2014,1,ATIVO,2014,1,3.0,0,Reprovado,0


In [70]:
# Count the status sum-up
disciplineStatus = pd.DataFrame(disciplineStatus.groupby(["status"])['Count'].count()).reset_index()

# Print the dataSet head
disciplineStatus.head()

,status,Count
0,ATIVO,3440
1,CANCELADO,1047
2,CONCLUIDO,20
3,FORMADO,10
4,FORMANDO,43


In [71]:
from bokeh.palettes import RdBu9

# Tools
TOOLS = 'box_zoom,box_select,crosshair,resize,reset,hover,save'

# Donut chart settings
d = Donut(disciplineStatus, label=['status', 'Count'], values='Count',
          text_font_size='10pt', hover_text='status', legend='top_left', 
          tools=TOOLS, background_fill_color="#E8DDCB", title='Discipline status', 
          color=RdBu9, plot_width=600, plot_height=600)

# Print the chart
output_notebook()
show(d)

Loading BokehJS ...